### Problem set 8
### Submitted by: Abu Huraira

1. What is the type of guess in the following program? Justify your answer.
```haskell
    guess p g = do
        s <- getLine
        x <- g s
        case p x of
            True -> return x
            _ -> guess p g
```
**Answer:**

* `guess` is a function that takes two arguments, `p` and `g`, and returns a value within a monadic context.

* `p` is a function that takes an argument of some type (unspecified) and returns a `Bool`.

3. `g` is a function that takes a string (`s`) and returns a value of some type (unspecified). 

4. The `do` notation is used, which implies that the function operates within a monadic context. In this case, it appears to be using the `IO` monad, as it interacts with the user through `getLine`.

Therefore, we can infer the type of `guess` to be:

```haskell
    guess :: (a -> Bool) -> (String -> IO a) -> IO a
```

2. Implement a program `shoutBack :: IO ()` that goes through an infinite loop of reading user input and printing it back in CAPS. Use `toUpper :: Char -> Char` to convert a single character to upper case. Remember that `type String = [Char]`.
```haskell
    import Data.Char (toUpper)
    shoutBack :: IO ()
```

In [31]:
import Data.Char (toUpper)

shoutBack :: IO ()
shoutBack = do
    putStrLn "Enter text (or 'exit' to quit):"
    input <- getLine
    if input == "exit"
        then putStrLn "Goodbye!"
        else do
            let shouting = map toUpper input
            putStrLn shouting
            shoutBack

-- NOTE: Please use GHCi to test this function.

3. Implement the following functions over IO:

(a) foreverIO :: IO a -> IO b — run a given program forever in an infinite loop:
```
>>> foreverIO (putStrLn "Hello!")
Hello!
Hello!
Hello!
...
```
**Solution:**

In [32]:
foreverIO :: IO a -> IO b
foreverIO action = do
    action
    foreverIO action

-- foreverIO (putStrLn "Hello!") 
-- CAUTION: This will run forever.

(b) `whenIO :: Bool -> IO () -> IO ()` — run a given program if a condition is satisfied;

**Solution:**

In [33]:
whenIO :: Bool -> IO () -> IO ()
whenIO condition action = if condition then action else return ()

shouldRun = True
whenIO shouldRun (putStrLn "Running because condition is True.")

shouldRun = False
whenIO shouldRun (putStrLn "Not running because condition is False.")

Line 2: Use when
Found:
if condition then action else return ()
Why not:
Control.Monad.when condition action

Running because condition is True.

(c) `maybeIO :: Maybe (IO a) -> IO (Maybe a)` — run a given program if there is one;

**Solution:**

In [34]:
maybeIO :: Maybe (IO a) -> IO (Maybe a)
maybeIO Nothing  = return Nothing
maybeIO (Just action) = do
    Just <$> action
    
maybeIO (Just (putStrLn "There is a program, hence running!"))
maybeIO Nothing

There is a program, hence running!
Just ()

Nothing

(d) `sequenceMaybeIO :: [IO (Maybe a)] -> IO [a]` — run a sequence of programs and collect all results of type `a`;

**Solution:**

In [35]:
sequenceIO :: [IO a] -> IO [a]
sequenceIO [] = return []
sequenceIO (action:actions) = do
    x <- action
    xs <- sequenceIO actions
    return (x:xs)

sequenceIO [putStrLn "Hello", putStrLn "World", putStrLn "!"]

Hello
World
!
[(),(),()]

(e) `whileJustIO :: (a -> IO (Maybe a)) -> a -> IO ()` — starting with an initial value of type a, apply a given function to run a program and either get Nothing and stop or get the next value and repeat;

**Solution:**

In [17]:
-- TODO

(f) `forStateIO_ :: s -> [a] -> (s -> a -> IO s) -> IO s` — starting with an initial state of type s, go over values in the list of type `[a]` from left to right, applying a step function
`s -> a -> IO s` to update intermediate state on every element; return the final state of type `s`:
```haskell
    verboseSnoc :: [Int] -> Int -> IO [Int]
    verboseSnoc xs x = do
        putStrL
        
```
```
    >>> forStateIO_ [] [1, 2, 3] verboseSnoc
    appending 1 to the end of []
    appending 2 to the end of [1]
    appending 3 to the end of [1,2]
    [1,2,3]
```
**Solution:**

4. Implement a polymorphic higher-order function iforIO_ that runs a program for each element and its index in a given list (using given function). Provide an explicit type signature for `iforIO_`.

```haskell
    example = do
        iforIO_ [1, 2] (\i n ->
            iforIO_ "ab" (\j c ->
                print ((i, j), replicate n c)))
```
```
    >>> example
    ((0,0),"a")
    ((0,1),"b")
    ((1,0),"aa")
    ((1,1),"bb")
```
**Solution:**

In [25]:
iforIO_ :: (Monad m) => [a] -> (Int -> a -> m b) -> m ()
iforIO_ lst f = go 0 lst
  where
    go _ []     = return ()
    go i (x:xs) = do
      _ <- f i x
      go (i + 1) xs

example :: IO ()
example = do
    iforIO_ [1, 2] (\i n ->
        iforIO_ "ab" (\j c ->
            print ((i, j), replicate n c)))

example

((0,0),"a")
((0,1),"b")
((1,0),"aa")
((1,1),"bb")